In [1]:
import backtest
import numpy as np
import pandas as pd

In [2]:
DATA_PATH = "./data/"
dat = pd.read_parquet(DATA_PATH + "eurusd.parquet")
dat.shape, dat.columns

((661062, 3), Index(['time', 'close', 'volume'], dtype='object'))

In [3]:
opt_trades = {}
dd_constraints = np.arange(10,100,10)
for dd_bps in dd_constraints:
    opt_trades[dd_bps] = backtest.target_optimal(
        df_price = dat.close,
        fee_bps = 3,
        dd_bps = dd_bps
    )
    
    print(f"Done optimizing labels with dd_bps constraint = {dd_bps}.")

Done optimizing labels with dd_bps constraint = 10.
Done optimizing labels with dd_bps constraint = 20.
Done optimizing labels with dd_bps constraint = 30.
Done optimizing labels with dd_bps constraint = 40.
Done optimizing labels with dd_bps constraint = 50.
Done optimizing labels with dd_bps constraint = 60.
Done optimizing labels with dd_bps constraint = 70.
Done optimizing labels with dd_bps constraint = 80.
Done optimizing labels with dd_bps constraint = 90.


In [4]:
stats = {} # trading statistics
for dd_bps in dd_constraints:
    trades_in = dat.copy()
    trades_in['S'] = 0
    trades_in['L'] = 0
    trades_in['C'] = 0
    trades_in.loc[opt_trades[dd_bps] == 0, 'S'] = 1
    trades_in.loc[opt_trades[dd_bps] == 1, 'L'] = 1
    trades_in.loc[opt_trades[dd_bps] == 2, 'C'] = 1
    
    b = backtest.BackTest(data = trades_in,
                          decimal_pip = 5,
                          threshold = 0.5,
                          fee_bps = 3)
    stats[dd_bps] = b.trade_stats()

In [5]:
stats[90]

{'annual_return': 1.513,
 'sharpe': 15.48,
 'sortino': 29.06,
 'std_deviation': 0.098,
 'total_return': 26.7109,
 'trade_count_long': 724,
 'trade_count_short': 699,
 'trade_count_total': 1423,
 'average_return_long': 185.0,
 'average_return_short': 191.0,
 'accuracy_long': 1.0,
 'accuracy_short': 1.0,
 'accuracy_total': 1.0,
 'max_drawdown': -0.011,
 'max_drawdown_days': 3}